# Create a generative adversarial network, building and training a GAN that can generate hand-written images of digits(0-9)

### Learning Objectives
1. Build the generator and discriminator components of a GAN from scratch.
2. Create generator and discriminator loss functions.
3. Train your GAN and visualize the generated images.

### Getting Started
Import useful packages and the dataset you will use to build and train your GAN. A visualizer function is there to investigate the images the GAN will create.

In [4]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) # Set for testing purposes, please do not change!

def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in a uniform grid.
    '''
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

### Generator
Create a function to make a single layer/block for the generator's neural network.Each block should include
a linear transformation to map to another shape, a batch normalization for stabilization 
and finally a non-linear activation function (ReLU) so the output can be transformed in complex ways.

In [13]:
# Define the generator block function
def get_generator_block(input_dim, output_dim):
    '''
    Function for returning a block of the generator's neural network given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a generator neural network layer, with a linear transformation
        followed by a batch normalization and then a relu activation
    '''
    return nn.Sequential(
        # Hint: Replace all of the "None" with the appropriate dimensions.
        # The documentation may be useful if you're less familiar with PyTorch:
        # https://pytorch.org/docs/stable/nn.html.
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True)
    )

In [22]:
# Verify the generator block function
def test_gen_block(in_features, out_features, num_test=1000):
    block = get_generator_block(in_features, out_features)
    
    # Check the three parts
    assert len(block) == 3
    assert type(block[0]) == nn.Linear
    assert type(block[1]) == nn.BatchNorm1d
    assert type(block[2]) == nn.ReLU
    
    # Check the output shape
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)
    assert tuple(test_output.shape) == (num_test, out_features)
    assert test_output.std() > 0.55
    assert test_output.std() < 0.65
    
test_gen_block(25, 12)
test_gen_block(15, 28)
print("Success!")

Success!


###### Now a generator class can be built. It will take 3 values:
    1. The noise vector dimension
    2. The image dimension
    3. The initial hidden dimension

Using these values, the generator will build a nn with 5 layers/blocks. Beginning with the noise vector ,the 
generator will apply non-linear transformations via the block function until the tensor is mapped to the size of
the image to be outputted(the same size as the real images from MNIST). The final layer needs doesnot need a 
normalization or activation function, but foes need to be scaled with a sigmoid function.

Finally a forward pass function is available that takes in a noise vector and generates an image of the output 
dimension using the nn.

In [29]:
class Generator(nn.Module):
    '''
    Generator Class
    Values:
        z_dim: the dimension of the noise vector, a scalar
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
            (MNIST images are 28 x 28 so that is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, z_dim=10, im_dim=784, hidden_dim=128):
        super(Generator, self).__init__()
        # Build the neural network
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim*8, im_dim),
            nn.Sigmoid()
        )
    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor, returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
        return self.gen(noise)
    
    # Needed for grading
    def get_gen(self):
        '''
        Returns:
            the sequential model
        '''
        return self.gen

In [31]:
# Verify the generator class

def test_generator(z_dim, im_dim, hidden_dim, num_test=10000):
    gen = Generator(z_dim, im_dim, hidden_dim).get_gen()
    
    # Check there are six modules in the sequential part
    assert len(gen) == 6
   
    test_input = torch.randn(num_test, z_dim)
    test_output = gen(test_input)
    
    # Check that the output shape is correct
    assert tuple(test_output.shape) == (num_test, im_dim)
    assert test_output.max() < 1, "Make sure to use a sigmoid"
    assert test_output.min() > 0, "Make sure to use a sigmoid"
    assert test_output.min() < 0.5, "Don't use a block in your solution"
    assert test_output.std() > 0.05, "Don't use batchnorm here"
    assert test_output.std() < 0.15, "Don't use batchnorm here"
    
test_generator(5, 10, 20)
test_generator(20, 8, 24)
print("Success!")
    

Success!


#### Noise
Create noise vector to be able to use the generator, as the noise vector z has the important role of making sure 
the images generated from the same class don't all look the same--think of it as a random seed. It is generated 
randomly using PyTorch by sampling random numbers from the normal distribution. Since multiple images will be 
processed per pass, all the noise vectors will be generated at once.
Note: Whenever a new tensor is created using torch.ones, torch.zeros, or torch.randn, it is either created on a 
    target device, e.g. torch.ones(3, 3, device=device), or move into the target device using torch.ones(3,3).to(device).
    In general, torch.ones_like and torch.zeros_like is used instead of torch.ones or torch.zeros where possible.

In [33]:
def get_noise(n_samples, z_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim),
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    '''
    # NOTE: To use this on GPU with device='cuda', make sure to pass the device
    # argument tot the function you use to generate the noise
    #### START CODE HERE ####
    noise = torch.randn(n_samples, z_dim, device=device)
    return noise
    #### END CODE HERE ####

In [35]:
# Verify the noise vector function
def test_get_noise(n_samples, z_dim, device='cpu'):
    noise = get_noise(n_samples, z_dim, device)
    
    # Make sure a normal distribution was used
    assert tuple(noise.shape) == (n_samples, z_dim)
    assert torch.abs(noise.std() - torch.tensor(1.0)) < 0.01
    assert str(noise.device).startswith(device)
    
test_get_noise(1000, 100, 'cpu')
if torch.cuda.is_available():
    test_get_noise(1000, 32, 'cuda')
print("Success!")

Success!
